Data Mining: Basic Concepts - Winter 2023/24
---------------
``` 
> University of Konstanz 
> Department of Computer and Information Science
> Maximilian T. Fischer, Frederik Dennig, Yannick Metz, Udo Schlegel
```
__Organize in teams of 2 people, return the exercise on time using ILIAS__

---

Assignment 9 in Python

- __Please put your names and student IDs here___:
    - _Lorenz Rückert_, _01/911915_
    - _Lennart Kasserra_, _01/1358216_

---

#### Exercise 1:  K-means 

Perform k-means clustering with k=3 using the Euclidean distance by hand. Assume that initial clusters centroids are:

* Cluster 1: Record # 1
* Cluster 2: Record # 5
* Cluster 3: Record # 10

Compute for every iteration the updated cluster prototypes and state to which cluster each point is assigned after each iteration. 

_Note: In this exercise, you don’t need to normalize the data._

|ID|X1|X2|X3|X4|X5|
|-----|--------|-----------|-----------------|--------------|----|
|1|0.58|0.26|0.38|0.73|0.65|
|2|0.50|0.21|0.34|0.61|0.61|
|3|0.92|0.80|0.70|0.60|0.61|
|4|0.51|0.29|0.37|0.54|0.69|
|5|0.18|0.23|0.92|0.25|0.26|
|6|0.12|0.23|0.18|0.30|0.25|
|7|0.90|0.85|0.75|0.60|0.63|
|8|0.01|0.16|0.15|0.28|0.31|
|9|0.44|0.25|0.35|0.59|0.64|
|10|0.91|0.83|0.81|0.55|0.46|

_(Hint: you can use the ``![](imagename.png)`` to include a image in a markdown cell. You can use this to include a hand written calculation. Do not forget to also upload the image in your final submission.)_

```
> Your Answer
```

---

#### Exercise 3: Expectation Maximization (EM) Algorithm 

The sklearn class `GaussianMixture`can be used to compute the EM algorithm. 

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.mixture import GaussianMixture

##### __(a) Cluster the `em-data.csv` data. Describe each step and reason your parameter choices.__

In [21]:
em_data = pd.read_csv('em-data.csv') # load the data

data = em_data.values # convert to a numpy array

gmm = GaussianMixture(n_components=4, random_state=0)  # create a gmm model and define number of clusters based on visual inspection below

gmm.fit(data) # fit the model to the data

labels = gmm.predict(data) 

##### __(b) Visualize your results in a 2D plot.__

In [22]:
# Visualize the clusters
plt.scatter(data[:, 0], data[:, 1], c=labels, cmap='viridis')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Gaussian Mixture Model Clustering')
plt.show()

#### Exercise 2: Partitioning Around Medoids (PAM) 

Load the `USArrests.csv` dataset and the following methods which can be used to compute PAM. Please install the `pyclustering` package. 

_(Hint: Use only the following imports)_

In [5]:
!pip install pyclustering
import pandas as pd 
import numpy as np 
from pyclustering.cluster.kmedoids import kmedoids
from scipy.spatial.distance import pdist, cdist, squareform
import matplotlib.pyplot as plt
from random import randrange

In [6]:
# Load dataset here

##### __(a) Standardize the columns of the "USArrests" dataset. Explain why we have to standardize the dataset?__

```
> Your Answer
```

##### __(b) Compute the PAM (`kmedoids`) using `k = 4`. List the cluster and cluster medoids. Describe and explain the results.__

```
> Your Answer
```

##### __(c) Define a Python method to compute the measure for the compactness of a PAM clustering. The parameters of the method is the data and the instance of the kmedoids class (`kmedoids_instance`).__

$$TD=\sum\limits^k_{i = 1} TD(C_i) \text{ and } TD(C) = \sum\limits_{p \in C} dist(p, m_c)$$

The medoid $m_c$ is the representative element of the cluster. 

_(Hint: Use the `pdist` or `cdist` method to compute the pairwise distance between $p$ and $m_c$)_

##### __(d) Use the newly defined function to compute TD of PAM $k = 3$.__

##### __(e) Compute the TD's for PAM with $k = 1, \dots, 10$ and plot the results. Explain the results of the plot.__

```
> Your Answer
```

---